In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
crash_df = pd.read_csv('/Users/gabriellabarrett/Desktop/ADS/FinalProject/Motor_Vehicle_Crashes.csv',dtype={'BOROUGH':str,'ZIP CODE':str})

In [5]:
crash_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,04/18/2020,19:30,BRONX,10452,40.837315,-73.923310,"(40.837315, -73.92331)",SHAKESPEARE AVENUE,ANDERSON AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4309853,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,06/18/2020,19:40,BROOKLYN,11205,40.696278,-73.974660,"(40.696278, -73.97466)",NaN,NaN,129 PARK AVENUE,...,NaN,NaN,NaN,NaN,4321231,Sedan,NaN,NaN,NaN,NaN
2,05/21/2020,16:30,QUEENS,11379,40.708690,-73.875374,"(40.70869, -73.875374)",69 ROAD,76 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4315381,Sedan,Sedan,NaN,NaN,NaN
3,06/23/2020,10:00,MANHATTAN,10011,40.736350,-73.993385,"(40.73635, -73.993385)",NaN,NaN,90 5 AVENUE,...,Unspecified,NaN,NaN,NaN,4322468,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,04/20/2020,14:20,BROOKLYN,11206,40.702892,-73.944220,"(40.702892, -73.94422)",MANHATTAN AVENUE,VARET STREET,NaN,...,Unspecified,Unspecified,Unspecified,NaN,4309717,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,Station Wagon/Sport Utility Vehicle,NaN


In [6]:
crash_df['CRASH DATE'] = pd.to_datetime(crash_df['CRASH DATE'])
crash_df['CRASH DATE'].sort_values(ascending=False)
crash_df['dow'] = pd.to_datetime(crash_df['CRASH DATE']).dt.dayofweek
crash_df['day'] = pd.to_datetime(crash_df['CRASH DATE']).dt.day
crash_df['year'] = pd.to_datetime(crash_df['CRASH DATE']).dt.year

In [10]:
crash_df = crash_df[(crash_df.year>2015) & (crash_df.year<2020)]

In [11]:
crash_df.isna().sum()

CRASH DATE                            0
CRASH TIME                            0
BOROUGH                          321370
ZIP CODE                         321550
LATITUDE                          83628
LONGITUDE                         83628
LOCATION                          83628
ON STREET NAME                   213318
CROSS STREET NAME                442821
OFF STREET NAME                  697832
NUMBER OF PERSONS INJURED            17
NUMBER OF PERSONS KILLED             31
NUMBER OF PEDESTRIANS INJURED         0
NUMBER OF PEDESTRIANS KILLED          0
NUMBER OF CYCLIST INJURED             0
NUMBER OF CYCLIST KILLED              0
NUMBER OF MOTORIST INJURED            0
NUMBER OF MOTORIST KILLED             0
CONTRIBUTING FACTOR VEHICLE 1      2835
CONTRIBUTING FACTOR VEHICLE 2    133453
CONTRIBUTING FACTOR VEHICLE 3    845726
CONTRIBUTING FACTOR VEHICLE 4    891633
CONTRIBUTING FACTOR VEHICLE 5    900671
COLLISION_ID                          0
VEHICLE TYPE CODE 1                7043


In [70]:
crash_df_clean = crash_df[crash_df['LONGITUDE'].notna()]
crash_df_clean = crash_df_clean[crash_df_clean['LATITUDE'].notna()]
crash_df_clean = crash_df_clean[crash_df_clean['ON STREET NAME'].notna()]


In [71]:
crash_df_clean.isna().sum()

CRASH DATE                            0
CRASH TIME                            0
BOROUGH                          251719
ZIP CODE                         251816
LATITUDE                              0
LONGITUDE                             0
LOCATION                              0
ON STREET NAME                        0
CROSS STREET NAME                208635
OFF STREET NAME                  617370
NUMBER OF PERSONS INJURED             9
NUMBER OF PERSONS KILLED             14
NUMBER OF PEDESTRIANS INJURED         0
NUMBER OF PEDESTRIANS KILLED          0
NUMBER OF CYCLIST INJURED             0
NUMBER OF CYCLIST KILLED              0
NUMBER OF MOTORIST INJURED            0
NUMBER OF MOTORIST KILLED             0
CONTRIBUTING FACTOR VEHICLE 1      1892
CONTRIBUTING FACTOR VEHICLE 2     82187
CONTRIBUTING FACTOR VEHICLE 3    576981
CONTRIBUTING FACTOR VEHICLE 4    609129
CONTRIBUTING FACTOR VEHICLE 5    615279
COLLISION_ID                          0
VEHICLE TYPE CODE 1                4471


In [104]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopandas import GeoSeries, GeoDataFrame
import geopandas as gpd

np.random.seed(1)
DPI = 100

path_nybb = gpd.datasets.get_path('nybb')
boros = GeoDataFrame.from_file(path_nybb)
boros.set_index('BoroCode', inplace=True)
boros.sort_index()
boros


,BoroName,Shape_Leng,Shape_Area,geometry
BoroCode,,,,
5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [92]:
gdf = geopandas.GeoDataFrame(
    crash_df_clean, crs="EPSG:2263",geometry=geopandas.points_from_xy(crash_df_clean.LONGITUDE, crash_df_clean.LATITUDE))

#Right CRS: EPSG:2263

In [128]:
geometry = [Point(xy) for xy in zip(crash_df_clean.LONGITUDE, crash_df_clean.LATITUDE)]
data_geo = gpd.GeoDataFrame(crash_df_clean, crs="EPSG:2263", geometry=geometry)
#######

In [79]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(crash_df_clean['LONGITUDE'], crash_df_clean['LATITUDE'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Creating a Geographic data frame 
gdf = gpd.GeoDataFrame(crash_df_clean, crs=crs, geometry=geometry)

/Users/gabriellabarrett/opt/anaconda3/envs/gabs_env/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [95]:
gdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,dow,day,year,geometry
184,2016-12-20,10:30,BRONX,10468,40.863785,-73.90028,"(40.863785, -73.90028)",EAST 190 STREET,JEROME AVENUE,NaN,...,3583642,Sedan,Sedan,NaN,NaN,NaN,1,20,2016,POINT (-73.90028 40.86379)
506,2016-12-11,22:00,NaN,NaN,40.696540,-73.72703,"(40.69654, -73.72703)",CROSS ISLAND PARKWAY,NaN,NaN,...,3577043,Sedan,Sedan,NaN,NaN,NaN,6,11,2016,POINT (-73.72703 40.69654)
1073,2019-02-07,13:30,NaN,NaN,40.715828,-74.01539,"(40.715828, -74.01539)",MURRAY STREET,NORTH END AVENUE,NaN,...,4076942,Sedan,Sedan,NaN,NaN,NaN,3,7,2019,POINT (-74.01539 40.71583)
1559,2019-02-28,16:00,BRONX,10461,40.847004,-73.83229,"(40.847004, -73.83229)",CROSBY AVENUE,WESTCHESTER AVENUE,NaN,...,4088555,Sedan,Sedan,NaN,NaN,NaN,3,28,2019,POINT (-73.83229 40.84700)
1798,2019-02-20,17:45,QUEENS,11415,40.711823,-73.83612,"(40.711823, -73.83612)",UNION TURNPIKE,PARK LANE,NaN,...,4084096,Sedan,Sedan,NaN,NaN,NaN,2,20,2019,POINT (-73.83612 40.71182)


In [111]:
nyc_boro = gpd.read_file('/Users/gabriellabarrett/Desktop/ADS/FinalProject/nybb_21a/nybb.shp')

In [112]:
nyc_boro

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,1,Manhattan,361611.823950,6.366038e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
1,2,Bronx,464650.051807,1.187245e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."
2,3,Brooklyn,729592.941749,1.934091e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
3,4,Queens,900087.667191,3.044042e+09,"MULTIPOLYGON (((1032452.015 154469.237, 103245..."
4,5,Staten Island,325956.009000,1.623757e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."


In [115]:
#boros.head()

In [105]:
geometry = [Point(xy) for xy in zip(data3.Longitude, data3.Latitude)]
data_geo = gpd.GeoDataFrame(data3, crs="EPSG:4326", geometry=geometry)

In [124]:
from geopandas.tools import sjoin

df = gpd.sjoin(gdf[['COLLISION_ID','geometry']], nyc_boro[['BoroName','geometry']], how='left', op='intersects')



ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

In [129]:
from geopandas.tools import sjoin
pointInPolys = sjoin(data_geo, nyc_boro, how='left', op='intersects')
pointInPolys.geometry()

ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

In [110]:
from geopandas.tools import sjoin
pointInPolys = sjoin(gdf, boroughs, how='left_df', op='intersects')
pointInPolys.geometry()

ValueError: 'right_df' should be GeoDataFrame, got <class 'method'>

In [85]:
!pip install rtree==0.8.3
!pip install pygeos

     |████████████████████████████████| 1.9 MB 5.2 MB/s eta 0:00:01


In [50]:
!pip install geopandas

     |████████████████████████████████| 994 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 19.3 MB 12.7 MB/s eta 0:00:01
  Using cached Shapely-1.7.1-cp38-cp38-macosx_10_9_x86_64.whl (1.0 MB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached cligj-0.7.1-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)


SyntaxError: invalid syntax (<ipython-input-53-34cf9667a6d2>, line 1)

In [60]:
from geopandas.tools import sjoin
pointInPolys = sjoin(gdf, boros[['BoroName', 'geometry']], how='left', op='intersects')
pointInPolys.head()

<ipython-input-60-3ea839b8c69a>:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:2263

  pointInPolys = sjoin(gdf, boros[['BoroName', 'geometry']], how='left', op='intersects')


ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

In [ ]:
for i in range(len(crash_df_clean)):
    




In [ ]:
# Check if p1 is within the polygon using the within function
In [4]: p1.within(poly)
Out[4]: True

# Check if p2 is within the polygon
In [5]: p2.within(poly)
Out[5]: False

In [29]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# gdf_locations = gpd.read_file('tempdata/tl_2016_us_zcta510.shp')
# data = {'Site 1': '31.336968, -109.560959',
#         'Site 2': '31.347745, -108.229963',
#         'Site 3': '32.277621, -107.734724',
#         'Site 4': '31.655494, -106.420484',
#         'Site 5': '30.295053, -104.014528'}
df_site = pd.DataFrame.from_dict(crash_df_clean, orient='index',columns=['geometry'])
df_site['loc'] = df_site['geometry'].apply(lambda x: list(map(float,x.split(','))))
df_site['loc'] = df_site['loc'].apply(lambda x: Point(x[1],x[0]))
gdf_site = gpd.GeoDataFrame(df_site,geometry=df_site['loc'],crs=boros.crs).drop(['loc'], axis=1)
print(gdf_site)

TypeError: 'numpy.ndarray' object is not callable